In [15]:
import os

# import llama-7b
from langchain_community.chat_models import ChatOllama
from langchain_community.utilities.sql_database import SQLDatabase
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit

# load llama, make sure it's running locally or change the model to the one you want to use
llm = ChatOllama(model="llama2")

base_dir = os.path.dirname(os.getcwd())
data_limit = 100_000

In [3]:
base_dir

'/Users/chasenuzum/Documents'

In [11]:

db = SQLDatabase.from_uri("sqlite:///" + base_dir + "/nfl_data/nfl.db")

In [16]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

In [18]:
agent_executor = create_sql_agent(llm, verbose=True, toolkit=toolkit)

In [19]:
agent_executor.invoke(
    "List the top 5 players with the most passing yards in the 2020 NFL season."
)



> Entering new SQL Agent Executor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Great, let's get started!

Action: sql_db_query
Input: SELECT SUM(passing_yards) FROM players WHERE season = '2020';`

In [8]:
# have llama summarize the wiki page
print(wiki_page)
summarizer_input = """Summarize the following wiki page: {wikipedia_page}. Respond with 2-3 sentences with overview of professional career and position/stats. Mention anything
related to fantasy football if applicable."""
llm_expert = """You are a fantasy football expert and answer user questions about the NFL and it's players.

You have the following information: {wiki}. Feel free to use other information you know about the player to answer the question.

You are asked to answer the following question: 'What do you think of {player_name} as a fantasy football player? Are they startable?"""
prompt = ChatPromptTemplate.from_template(summarizer_input)
expert_prompt = ChatPromptTemplate.from_template(llm_expert)
# create a chain
chain_summarizer = prompt | llm | StrOutputParser()
chain_expert = {"wiki": chain_summarizer, "player_name": itemgetter("player_name")} | \
expert_prompt | llm | StrOutputParser()

Page: Josh Morgan
Summary: Joshua Lewis Morgan (born June 20, 1985) is a former American football wide receiver. He was selected by the San Francisco 49ers in the sixth round of the 2008 NFL Draft.  He played college football at Virginia Tech. He also played for the Washington Redskins and Chicago Bears.

Page: Josh Morgan (politician)
Summary: Joshua Douglas Morgan is a Canadian politician who has been serving as the 65th mayor of London since 2022. He was elected mayor in the 2022 London municipal election.

Page: The Subways
Summary: The Subways are an English rock band from Welwyn Garden City, Hertfordshire. Formed in 2002, the band consists of Billy Lunn (guitar, lead vocals), Charlotte Cooper (bass, lead vocals), and Camille Phillips (drums). Founding member and original drummer Josh Morgan left the band for personal reasons in October 2020.
The Subways' debut album, Young for Eternity, was released on 4 July 2005 in the UK and 14 February 2006 in the U.S. Their second album, All